In [1]:
import numpy as np
from scipy.interpolate import lagrange
from scipy.optimize import OptimizeResult, minimize_scalar

def good_bracket(func, bracket):
    a, c, b = bracket
    return (func(a) > func(c)) and (func(b) > func(c))


def parabolic_step(f, args, bracket, **options):
    stop = False
    funcalls = 0
    niter = 0
    while not stop:
        niter += 1
        interpolator = lagrange(np.array(bracket), f(np.array(bracket)))
        funcalls += 3
        a, b, c = interpolator.coeffs
        d = -0.5*b/a
        if np.allclose(bracket[1], d):
            minima = d
            stop = True
        elif bracket[1] < d:
            newbracket = [bracket[1], d, bracket[2]]
            if good_bracket(f, newbracket):
                bracket = newbracket
            else:
                bracket = [bracket[0], bracket[1], d]
        else:
            newbracket = [d, bracket[1], bracket[2]]
            if good_bracket(f, newbracket):
                bracket = newbracket
            else:
                bracket = [bracket[0], d, bracket[1]]
    return OptimizeResult(fun=f(minima), x=minima, nit=niter, nfev=funcalls)

In [2]:
def g(x): return -np.exp(-x)*np.sin(x)

f = np.vectorize(lambda x: max(1-x, 2+x))

print good_bracket(f, [-1, -0.5, 1])

print minimize_scalar(f, bracket=[-1, -0.5, 1], method=parabolic_step)

True
  fun: array(1.5000021457670878)
 nfev: 33
  nit: 11
    x: -0.50000214576708779


In [3]:
print good_bracket(g, [0, 1.2, 1.5])

print minimize_scalar(g, bracket=[0,1.2,1.5], method=parabolic_step)

True
  fun: -0.32239694192707452
 nfev: 54
  nit: 18
    x: 0.78540558549352946


In [4]:
minimize_scalar(f, method='brent', bracket=[-1, -0.5, 1])

     fun: array(1.5)
    nfev: 22
     nit: 21
 success: True
       x: -0.5

In [5]:
minimize_scalar(f, method='golden', bracket=[-1, -0.5, 1])

  fun: array(1.5)
 nfev: 44
    x: -0.5

In [6]:
minimize_scalar(g, method='brent', bracket=[0, 1.2, 1.5])

     fun: -0.32239694194483443
    nfev: 11
     nit: 10
 success: True
       x: 0.78539817181020977

In [7]:
minimize_scalar(g, method='golden', bracket=[0, 1.2, 1.5])

  fun: -0.32239694194483448
 nfev: 43
    x: 0.7853981573284226

In [8]:
minimize_scalar(g, method='bounded', bounds=(0, 1.5))

     fun: -0.3223969419448342
 message: 'Solution found.'
    nfev: 10
  status: 0
 success: True
       x: 0.78539813414162596